<a href="https://colab.research.google.com/github/MRazin172/2348534_LLM_Project/blob/main/2348534_LLM_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers
!pip install ctransformers
!pip install pinecone-client
!pip install langchain
!pip install pypdf
!pip install python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found 

In [ ]:
!pip install langchain_community
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.1 MB/s eta 0:00:00


In [ ]:
model_name_or_path = "TheBloke/Llama-2-7B-Chat-GGML"
model_basename = "llama-2-7b-chat.ggmlv3.q4_0.bin" # the model is in bin format

In [ ]:
from huggingface_hub import hf_hub_download
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-7b-chat.ggmlv3.q4_0.bin:   0%|          | 0.00/3.79G [00:00<?, ?B/s]

In [ ]:
book_link = "https://ia804601.us.archive.org/30/items/GaleEncyclopediaOfAlternativeMedicine2ndEdition.Vol.1AC/Gale%20Encyclopedia%20of%20Alternative%20Medicine,%202nd%20Edition.%20Vol.%201,%20A-C.pdf"

In [ ]:
import os
import requests

# Create a 'pdfs' folder if it doesn't exist
if not os.path.exists('data'):
    os.makedirs('data')

# URL of the PDF you want to download
pdf_url = book_link

# Get the filename from the URL
filename = pdf_url.split("/")[-1]

# Full path where the PDF will be saved
save_path = os.path.join('data', filename)

# Download the PDF
response = requests.get(pdf_url)

# Check if the request was successful
if response.status_code == 200:
    # Write the content to a file
    with open(save_path, 'wb') as file:
        file.write(response.content)
    print(f"PDF downloaded and saved to {save_path}")
else:
    print(f"Failed to download PDF. Status code: {response.status_code}")

PDF downloaded and saved to data/Gale%20Encyclopedia%20of%20Alternative%20Medicine,%202nd%20Edition.%20Vol.%201,%20A-C.pdf


In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [ ]:
PINECONE_API_KEY="b8e2019b-4cc3-4f53-865e-eb43f7e35e5d"
PINECONE_API_ENV="starter"

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader


In [ ]:
loader = PyPDFDirectoryLoader("data")


In [ ]:
!pip install pypdf


In [ ]:
extracted_data = loader.load()


In [ ]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 6918


In [ ]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
embeddings = download_hugging_face_embeddings()


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
index_name="medicure-chatbot"


In [ ]:
!pip install --upgrade langchain-pinecone


In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
    api_key=PINECONE_API_KEY
)
index = pc.Index(index_name)

In [ ]:
embedded_texts = embeddings.embed_documents([t.page_content for t in text_chunks])

# Prepare vectors for upsert
vectors_to_upsert = []
for i, (chunk, embedding) in enumerate(zip(text_chunks, embedded_texts)):
    vector = {
        "id": f"chunk_{i}",
        "values": embedding,
        "metadata": {
            "text": chunk.page_content,
            # Add any other metadata you want to include
        }
    }
    vectors_to_upsert.append(vector)

# Function to split list into chunks
def chunk_list(lst, chunk_size):
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

# Split vectors into smaller batches
batch_size = 100  # You might need to adjust this
batches = chunk_list(vectors_to_upsert, batch_size)

# Upsert batches to Pinecone
for i, batch in enumerate(batches):
    try:
        index.upsert(
            vectors=batch,
            namespace="ns1"  # Replace with your desired namespace
        )
        print(f"Batch {i+1}/{len(batches)} upserted successfully")
    except Exception as e:
        print(f"Error upserting batch {i+1}: {str(e)}")
        # You might want to implement retry logic here

print("Upsert completed")

Batch 1/70 upserted successfully
Batch 2/70 upserted successfully
Batch 3/70 upserted successfully
Batch 4/70 upserted successfully
Batch 5/70 upserted successfully
Batch 6/70 upserted successfully
Batch 7/70 upserted successfully
Batch 8/70 upserted successfully
Batch 9/70 upserted successfully
Batch 10/70 upserted successfully
Batch 11/70 upserted successfully
Batch 12/70 upserted successfully
Batch 13/70 upserted successfully
Batch 14/70 upserted successfully
Batch 15/70 upserted successfully
Batch 16/70 upserted successfully
Batch 17/70 upserted successfully
Batch 18/70 upserted successfully
Batch 19/70 upserted successfully
Batch 20/70 upserted successfully
Batch 21/70 upserted successfully
Batch 22/70 upserted successfully
Batch 23/70 upserted successfully
Batch 24/70 upserted successfully
Batch 25/70 upserted successfully
Batch 26/70 upserted successfully
Batch 27/70 upserted successfully
Batch 28/70 upserted successfully
Batch 29/70 upserted successfully
Batch 30/70 upserted su

In [ ]:
query = "What are piles"

# Generate embedding for the query
query_embedding = embeddings.embed_query(query)

# Perform similarity search
search_results = index.query(
    namespace="ns1",  # Replace with your actual namespace
    vector=query_embedding,
    top_k=1,  # Number of results you want
    include_values=True,
    include_metadata=True
)

# Process and print results
print("Results:")
for match in search_results['matches']:
    print(f"ID: {match['id']}")
    print(f"Score: {match['score']}")
    print(f"Metadata: {match['metadata']}")
    print("---")

Results:
ID: chunk_6492
Score: 0.325279444
Metadata: {'text': 'nence, usually occurring on the feet and toes. It mani-fests as a rough and painful bump. A callus is a rough,thickened lump of dead skin that is usually painless. Itmay be found on the feet, the hands, or anywhere elsethere is repeated friction and pressure.\nDescription\nCorns and calluses affect about 5% of the popula-'}
---
